###Ingest driver.json file

####Step 1 - Read the JSON file using the spark dataframe reader 

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename",StringType(), True),
                                 StructField("surname",StringType(), True)])


In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                   StructField("driverRef", StringType(), True),
                                   StructField("number", IntegerType(), True),
                                   StructField("code", StringType(), True),
                                   StructField("name", name_schema),
                                   StructField("dob", DateType(), True),
                                   StructField("nationality", StringType(), True),
                                   StructField("url", StringType(), True)])

In [0]:
driver_df = spark.read \
.schema(driver_schema)\
.json("/mnt/formula1courseds/raw/drivers.json")

In [0]:
driver_df.printSchema()

In [0]:
display(driver_df)

#### Step 2 - Rename columns and add new columns
1. driverId renamed to driver_id
2. driverRef renamed to driver_ref
3. ingestion date added
4. name added with concatenation of forename and surname  

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
driver_with_columns_df = driver_df.withColumnRenamed("driverId", "driver_id") \
                                   .withColumnRenamed("driverRef", "driver_ref") \
                                   .withColumn("ingestion_date", current_timestamp()) \
                                   .withColumn("name",concat(col("name.forename"),lit(" "),col("name.surname")))

In [0]:
display(driver_with_columns_df)

#### Step 4 - Drop unwanted columns

In [0]:
driver_final_df = driver_with_columns_df.drop("url")

In [0]:
display(driver_final_df)

#### Step 4 - Write to output to processed container in parquet format

In [0]:
driver_final_df.write.mode("overwrite").parquet("/mnt/formula1courseds/processed/drivers")

In [0]:
#display(spark.read.parquet("/mnt/formula1courseds/processed/drivers"))